In [84]:
import pandas as pd

In [79]:
FILE_PATH = "test_2k_output_Feb_26_2020_output_Feb_18_2021.tsv"

def read_examples_from_file(file_path):
    texts = []
    te_label_sequences = []
    pe_label_sequences = []
    relations = [] # dictionary, key = pair of token id span <ASPECT_SPAN, OPINION_SPAN>, value = sentiment polarity of the relation
    with open(file_path) as f:
        count_sentence = 0

        tokens = []
        te_labels = []
        pe_labels = []
        relation_labels = []
        for line in f:
            line = line.strip()
            if not line and tokens: # empty line                    
                # convert relation to dict
                new_relation_labels = {}
                for relation_label in relation_labels:
                    relation_polarity = relation_label.split('[')[0]
                    ids = relation_label.split('[')[1]

                    ate_label_id = ids.split('_')[0]
                    ote_label_id = ids.split('_')[1][:-1]

                    ate_label_idx = []
                    for i in range(len(te_labels)):
                        if '_' not in te_labels[i]:
                            label_id = te_labels[i].split('[')[1][:-1]
                            if ate_label_id == label_id:
                                ate_label_idx.append(i)

                    ote_label_idx = []
                    for i in range(len(te_labels)):
                        if '_' not in te_labels[i]:
                            label_id = te_labels[i].split('[')[1][:-1]
                            if ote_label_id == label_id:
                                ote_label_idx.append(i)

                    new_relation_labels[(f'{ote_label_idx[0]}-{ote_label_idx[-1]}', f'{ate_label_idx[0]}-{ate_label_idx[-1]}')] = relation_polarity

                # convert te label to iob
                prev_label_id = -1
                new_te_labels = []
                for te_label in te_labels:
                    if te_label == '_':
                        new_te_labels.append('O')
                    else:
                        label = te_label.split('[')[0]
                        label_id = te_label.split('[')[1][:-1]
                        if label_id == prev_label_id:
                            new_te_labels.append(f'I-{label}')
                        else:
                            new_te_labels.append(f'B-{label}')
                        prev_label_id = label_id

                # convert pe label to iob
                new_pe_labels = []
                for pe_label in pe_labels:
                    label = pe_label.split('[')[0]
                    label_id = pe_label.split('[')[1][:-1]
                    new_pe_labels.append(label)

                texts.append(tokens)
                te_label_sequences.append(new_te_labels)
                pe_label_sequences.append(new_pe_labels)
                relations.append(new_relation_labels)

                tokens = []
                te_labels = []
                pe_labels = []
                relation_labels = []
                count_sentence += 1

            if count_sentence == 5:
                break

            line = line.split('\t')
            if len(line) == 8:
                token = line[2]
                te_and_relation_labels = line[3].split('|')
                pe_label = line[4]
                if len(te_and_relation_labels) > 1:
                    te_label = te_and_relation_labels[0] if ('ASPECT' or 'SENTIMENT') in te_and_relation_labels[0] else te_and_relation_labels[1]
                    relation_label = te_and_relation_labels[1] if ('ASPECT' or 'SENTIMENT') in te_and_relation_labels[0] else te_and_relation_labels[0]
                    relation_labels.append(relation_label)
                else:
                    te_label = te_and_relation_labels[0]

                tokens.append(token)
                te_labels.append(te_label)
                pe_labels.append(pe_label)
    
    return texts, te_label_sequences, pe_label_sequences, relations

In [80]:
texts, te_label_sequences, pe_label_sequences, relations = read_examples_from_file(FILE_PATH)